In [1]:
import os
import numpy as np
import PIL
from PIL import Image
import cv2

In [ ]:
def segmentation(ann_dir,category_dir):
    pairs=set()
    for ann_file in os.listdir(ann_dir):
        ann_path=os.path.join(ann_dir,ann_file)
        with open(ann_path,'r') as file:
            for line in file:
                line=line.strip().split(',')
                category= line[7]
                target_id=line[1]
                pairs.add((category,target_id))

    
    for category,target_id in pairs:
        lines_out=[]
        for ann_file in os.listdir(ann_dir):
            ann_path=os.path.join(ann_dir,ann_file)
            with open(ann_path,'r') as file :
                for line in file:
                    line=line.strip().split(',')
                    if line[7]==category and line[1]==target_id:
                        lines_out.append(line)
        
        category_folder=os.path.join(category_dir,f"category_{category}")
        os.makedirs(category_folder, exist_ok=True)
        target_folder=os.path.join(category_folder,f"target_{target_id}")
        os.makedirs(target_folder, exist_ok=True)

        target_path=os.path.join(target_folder,f"{target_id}.txt")
        with open(target_path,'w') as file:
            for line in lines_out:
                file.write(','.join(line)+'\n')
                

segmentation(
        ann_dir='D:/ML/iitiSoC/PS 3/visdrone/videos/VisDrone2019-VID-train/annotations',
        category_dir='D:/ML/iitiSoC/PS 3/visdrone/videos/VisDrone2019-VID-train/categories'

    )

segmentation(
        ann_dir='D:/ML/iitiSoC/PS 3/visdrone/videos/VisDrone2019-VID-val/annotations',
        category_dir='D:/ML/iitiSoC/PS 3/visdrone/videos/VisDrone2019-VID-val/categories'
)

In [ ]:
def visdrone_to_yolo(category_dir,ann_dir,sequence_dir):
    
    for category_folder in os.listdir(category_dir):
        category_path=os.path.join(category_dir,category_folder)
        for target_folder in os.listdir(category_path):
            target_path =os.path.join(category_path, target_folder)
            for file in os.listdir(target_path):
                file_path=os.path.join(target_path,file)
                lines_out =[]
                for ann_file in os.listdir(ann_dir):
                    sequence_name = ann_file.replace('.txt', '')
                sequence_path = os.path.join(sequence_dir, sequence_name)
                with open(file_path,'r') as f:
                    for line in f:
                        line=line.strip().split(',')
                        frame_id,target_id,x,y,width,height,score,category,truncation,occlusion=map(float,line[:10])
                        frame_id=int(frame_id)
                        target_id=int(target_id)
                        cls=int(category)-1
                        if cls<0:
                            continue
                        frame_name = f"{frame_id:07d}.jpg"
                        frame_path = os.path.join(sequence_path, frame_name)
                        if not os.path.exists(frame_path):
                            continue
                        frame= Image.open(frame_path)
                        w,h=frame.size
                        x_center= (x + width / 2) / w
                        y_center= (y + height / 2) / h
                        width_norm=width / w
                        height_norm=height / h
                        lines_out.append(f"{cls} {x_center} {y_center} {width_norm} {height_norm}\n")
                output_file = os.path.join(target_path, f"{target_folder}.txt")
                with open(output_file,'w') as f:
                    f.writelines(lines_out)

visdrone_to_yolo(
        category_dir='D:/ML/iitiSoC/PS 3/visdrone/videos/VisDrone2019-VID-train/categories',
        ann_dir='D:/ML/iitiSoC/PS 3/visdrone/videos/VisDrone2019-VID-train/annotations',
        sequence_dir='D:/ML/iitiSoC/PS 3/visdrone/videos/VisDrone2019-VID-train/sequences'
)

visdrone_to_yolo(
        category_dir='D:/ML/iitiSoC/PS 3/visdrone/videos/VisDrone2019-VID-val/categories',
        ann_dir='D:/ML/iitiSoC/PS 3/visdrone/videos/VisDrone2019-VID-val/annotations',
        sequence_dir='D:/ML/iitiSoC/PS 3/visdrone/videos/VisDrone2019-VID-val/sequences'
)


resizing the frames to 640 x 640 

In [ ]:
def resizing(seq_dir):
    for sequence in os.listdir(seq_dir):
        sequence_path =os.path.join(seq_dir,sequence)
        for frame in os.listdir(sequence_path):
            frame_path = os.path.join(sequence_path, frame)
            img = cv2.imread(frame_path)
            img = cv2.resize(img, (928, 928))
            cv2.imwrite(frame_path, img)
resizing(
        seq_dir='D:/ML/iitiSoC/PS 3/visdrone/videos/VisDrone2019-VID-train/sequences'
        )
resizing(
        seq_dir='D:/ML/iitiSoC/PS 3/visdrone/videos/VisDrone2019-VID-val/sequences'
)


In [2]:
def create_visdrone_yaml(video_dir,train_dir,val_dir,yaml_path):
    yaml_content = f"""\
path: {video_dir}
train: {train_dir}
val: {val_dir}

# Class names (10 classes)
names:
  0: pedestrian
  1: people
  2: bicycle
  3: car
  4: van
  5: truck
  6: tricycle
  7: awning-tricycle
  8: bus
  9: motor

# Class count
nc: 10
"""
    
    with open(yaml_path, 'w') as f:
        f.write(yaml_content)
    print(f"YAML config created: {yaml_path}")

create_visdrone_yaml(
    video_dir='D:/ML/iitiSoC/PS 3/visdrone/videos',
    train_dir='D:/ML/iitiSoC/PS 3/visdrone/videos/VisDrone2019-VID-train',
    val_dir='D:/ML/iitiSoC/PS 3/visdrone/videos/VisDrone2019-VID-val',
    yaml_path='D:/ML/iitiSoC/PS 3/visdrone/videos/visdrone.yaml'
)


YAML config created: D:/ML/iitiSoC/PS 3/visdrone/videos/visdrone.yaml


In [3]:
%pip install ultralytics
from ultralytics import YOLO 

Note: you may need to restart the kernel to use updated packages.


In [6]:
model = YOLO("yolov8s.pt")
model.train(
    data=r'D:/ML/iitiSoC/PS 3/visdrone/videos/visdrone.yaml',
    epochs=5)
    




Ultralytics 8.3.159  Python-3.13.3 torch-2.7.1+cpu CPU (13th Gen Intel Core(TM) i5-1335U)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=D:/ML/iitiSoC/PS 3/visdrone/videos/visdrone.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train18, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=

train: Scanning D:\ML\iitiSoC\PS 3\visdrone\videos\VisDrone2019-VID-train\sequences\uav0000013_00000_v.cache... 0 images, 24201 backgrounds, 0 corrupt: 100%|██████████| 24201/24201 [00:00<?, ?it/s]

WARNING Labels are missing or empty in D:\ML\iitiSoC\PS 3\visdrone\videos\VisDrone2019-VID-train\sequences\uav0000013_00000_v.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access  (ping: 0.00.0 ms, read: 416.1100.8 MB/s, size: 379.9 KB)


d:\ML\iitiSoC\PS 3\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning D:\ML\iitiSoC\PS 3\visdrone\videos\VisDrone2019-VID-val\sequences\uav0000086_00000_v.cache... 0 images, 2846 backgrounds, 0 corrupt: 100%|██████████| 2846/2846 [00:00<?, ?it/s]

WARNING Labels are missing or empty in D:\ML\iitiSoC\PS 3\visdrone\videos\VisDrone2019-VID-val\sequences\uav0000086_00000_v.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.



d:\ML\iitiSoC\PS 3\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Plotting labels to runs\detect\train18\labels.jpg... 
WARNING zero-size array to reduction operation maximum which has no identity
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train18
Starting training for 5 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/1513 [00:03<?, ?it/s]


KeyboardInterrupt: 